In [17]:
sc.install_pypi_package("pandas==0.25.1")
# sc.install_pypi_package("tensorflow_hub")

# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read CSV File and Get DataFrame (pyspark)

In [2]:
import pyspark.sql.functions as F
import re
from pyspark.sql.types import *

train_path = "s3://myprojectsentiment/data/train.csv"
test_path = "s3://myprojectsentiment/data/test.csv"

def readCSV(path):
    df = (spark.read
      .option("charset", "utf-8")
      .option("multiline", "true")
      .option("quote", '"')
      .option("header", "true")
      .option("escape", "\\")
      .option("escape", '"')
      .csv(path))
    df = df.select([F.col(col).alias(re.sub("[^0-9a-zA-Z$]+","",col)) for col in df.columns])
    df = df.withColumn('label', F.col('label').cast(DoubleType()))
    return df

train_df = readCSV(train_path)
test_df = readCSV(test_path)

train_df.printSchema()
train_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- content: string (nullable = true)
 |-- label: double (nullable = true)

+--------------------+-----+
|             content|label|
+--------------------+-----+
|Working with one ...|  0.0|
|Well...tremors I,...|  0.0|
|Ouch! This one wa...|  0.0|
|I've seen some cr...|  0.0|
|"Carriers" follow...|  0.0|
|I had been lookin...|  0.0|
|Effect(s) without...|  0.0|
|This picture star...|  0.0|
|I chose to see th...|  0.0|
|This film has to ...|  0.0|
|I felt brain dead...|  0.0|
|A young scientist...|  0.0|
|Inept, boring, an...|  0.0|
|From the first ti...|  0.0|
|I find it hard to...|  0.0|
|I actually saw Ch...|  0.0|
|I went to school ...|  0.0|
|I haven't seen th...|  0.0|
|I haven't seen an...|  0.0|
|One would think t...|  0.0|
+--------------------+-----+
only showing top 20 rows

In [3]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline# Start Spark Session with Spark NLP
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, SQLTransformer,IndexToString
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator# convert text column to spark nlp document
from pyspark.ml.classification import RandomForestClassifier

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import feature as spark_ft
from pyspark.ml.pipeline import PipelineModel
import re
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Tool Function

* Get Best Param from CV
* Save model
* Load Model

In [21]:
def printBestParams(cvModel):
    hyperparams = cvModel.getEstimatorParamMaps()[np.argmax(cvModel.avgMetrics)]
    hyper_list = []

    for i in range(len(hyperparams.items())):
        hyper_name = re.search("name='(.+?)'", str([x for x in hyperparams.items()][i])).group(1)
        hyper_value = [x for x in hyperparams.items()][i][1]

        hyper_list.append({hyper_name: hyper_value})

    print(hyper_list)
    
    
def savemodel(bestModel,mPath):
    bestModel.write().overwrite().save(mPath)

    
def readmodel(mPath,test_df):
    persistedModel = PipelineModel.load(mPath)
    # predict
    predictionsDF = persistedModel.transform(test_df)
    return predictionsDF

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Preprocess PipeLine 

I use Sparknlp to do the preprocess, detail info can be seen here :https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/2.Text_Preprocessing_with_SparkNLP_Annotators_Transformers.ipynb

In [6]:
document_assembler = DocumentAssembler() \
    .setInputCol("content") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentences')

# convert document to array of tokens
tokenizer = Tokenizer() \
  .setInputCols(["sentences"]) \
  .setOutputCol("token")
 
# clean tokens 
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"]) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

# remove stopwords
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

# stems tokens to bring it to root form
stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

tokenassembler = TokenAssembler()\
    .setInputCols(["sentences", "stem"]) \
    .setOutputCol("clean_text")

# Convert custom document structure to array of tokens.
finisher = Finisher() \
    .setInputCols(["stem"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

preprocess_pipeline = Pipeline(
    stages=[document_assembler, 
            sentenceDetector,
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            stemmer, 
            tokenassembler,
            finisher
           ])

preprocess_model = preprocess_pipeline.fit(train_df)
processed_train = preprocess_model.transform(train_df).persist()
processed_test = preprocess_model.transform(test_df).persist()

print(processed_train.count())
print(processed_test.count())

processed_train.select('content','label','clean_text','token_features').show(5)
preprocess_model.write().overwrite().save('s3://myprojectsentiment/preproces/preprocess_model')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read processed content

In [22]:
processed_train = readmodel('s3://myprojectsentiment/preproces/preprocess_model', train_df)
processed_test = readmodel('s3://myprojectsentiment/preproces/preprocess_model', test_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Feature Extracion

### Convert to TF IDF model

- **Term-Frequency Inverse Document Frequency (TF-IDF)**


In [8]:
# To generate Term Frequency , 1500 is selected by cv
tf = HashingTF(inputCol="token_features", outputCol="tf", numFeatures=1500)
# To generate Inverse Document Frequency
idf = IDF(inputCol="tf", outputCol="features", minDocFreq=5)

feature_pipeline = Pipeline(stages=[tf, idf])
feature_model = feature_pipeline.fit(processed_train)

tran_data = feature_model.transform(processed_train).persist()
test_data = feature_model.transform(processed_test).persist()

tran_data.select('content','label','features').show(5)
# tran_data.printSchema()

feature_model.write().overwrite().save('s3://myprojectsentiment/preproces/tf_model')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Word2vec

In [9]:
# text2vec = spark_ft.Word2Vec(
#     vectorSize=50, minCount=5, seed=123, 
#     inputCol='token_features', outputCol='text_vec', 
#     windowSize=5, maxSentenceLength=30
# )
# assembler = spark_ft.VectorAssembler(inputCols=['text_vec'], outputCol='features')

# feature_pipeline = Pipeline(stages=[text2vec,assembler])
# feature_model2 = feature_pipeline.fit(processed_train)

# tran = feature_model2.transform(processed_train).persist()
# test = feature_model2.transform(processed_test).persist()

# tran.select('content','label','features').show(5)
# feature_model2.write().overwrite().save('s3://myprojectsentiment/preproces/word2vec_model')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Read processed feature

In [23]:
tf_path = 's3://myprojectsentiment/preproces/tf_model'
vec_path = 's3://myprojectsentiment/preproces/word2vec_model'

train = readmodel(tf_path, processed_train).select('content','label','features')
test = readmodel(tf_path, processed_test).select('content','label','features')

# tranvc = readmodel(vec_path, processed_train).select('content','label','features')
# testvc = readmodel(vec_path, processed_test).select('content','label','features')

train.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+
|             content|label|            features|
+--------------------+-----+--------------------+
|Working with one ...|  0.0|(1500,[121,186,24...|
|Well...tremors I,...|  0.0|(1500,[0,1,15,29,...|
|Ouch! This one wa...|  0.0|(1500,[14,56,83,9...|
|I've seen some cr...|  0.0|(1500,[7,10,12,14...|
|"Carriers" follow...|  0.0|(1500,[0,15,21,29...|
+--------------------+-----+--------------------+
only showing top 5 rows

# Classification Models

## Naive Bayes Multinomial with Cross-validation

In [ ]:
# To generate Term Frequency
hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures")
# To generate Inverse Document Frequency
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

# create the trainer and set its parameters
nb = NaiveBayes(modelType="multinomial")

# this grid will have 2*2 = 4 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [1200, 1500]) \
    .addGrid(nb.smoothing, [0.1, 0.01]) \
    .build()

pipeline = Pipeline(stages=[hashingTF, idf, nb])

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(processed_train)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(processed_test)
selected = prediction.select( "content", "label", "prediction")
selected.show(5)

savemodel(cvModel.bestModel,"s3://myprojectsentiment/train_model/NB_tf")
printBestParams(cvModel)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Apply the best Params and predict

In [12]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
selected = predictions.select( "content", "label", "prediction")
selected.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+----------+
|             content|label|prediction|
+--------------------+-----+----------+
|Alan Rickman & Em...|  0.0|       0.0|
|I have seen this ...|  0.0|       1.0|
|In Los Angeles, t...|  0.0|       0.0|
|This film is bund...|  0.0|       0.0|
|I only comment on...|  0.0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows

### NB Model Evaluate

#### AUC-ROC 

In [13]:
# compute accuracy on the test set
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(selected)

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g " % (1.0 - accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy = 0.79236
Test Error = 0.20764

#### Precision-Recall

In [14]:
# compute accuracy on the test set
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderPR")
accuracy = evaluator.evaluate(selected)

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g " % (1.0 - accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy = 0.769274
Test Error = 0.230726

## Logist Regression with Cross-validation

In [ ]:
# To generate Term Frequency
hashingTF = HashingTF(inputCol="token_features", outputCol="rawFeatures")

# To generate Inverse Document Frequency
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

lr = LogisticRegression(maxIter=10)

# this grid will have 3*3 = 9 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(hashingTF.numFeatures, [1000, 1500, 2000]) \
    .addGrid(lr.regParam, [0.1, 0.08, 0.03]) \
    .build()

pipeline = Pipeline(stages=[ hashingTF,idf, lr])

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(processed_train)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(processed_test)
selected = prediction.select("content", "label", "prediction")
selected.show(5)

In [ ]:
printBestParams(cvModel)

savemodel(cvModel.bestModel,'s3://myprojectsentiment/train_model/LR_tf')

### Apply the best Params and predict

In [16]:
lr = LogisticRegression(maxIter=10,regParam=0.1)

# train the model
model = lr.fit(train)

# select example rows to display.
predictions = model.transform(test)
selected = predictions.select( "content", "label", "prediction")
selected.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+----------+
|             content|label|prediction|
+--------------------+-----+----------+
|Alan Rickman & Em...|  0.0|       0.0|
|I have seen this ...|  0.0|       1.0|
|In Los Angeles, t...|  0.0|       0.0|
|This film is bund...|  0.0|       0.0|
|I only comment on...|  0.0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows

### LR Model Evaluate

#### AUC-ROC 

In [17]:
# compute accuracy on the test set
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(selected)

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g " % (1.0 - accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy = 0.82776
Test Error = 0.17224

#### Precision-Recall

In [18]:
# compute accuracy on the test set
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderPR")
accuracy = evaluator.evaluate(selected)

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g " % (1.0 - accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy = 0.799082
Test Error = 0.200918

## Random Forest with CV

In [20]:
rf = RandomForestClassifier()

# this grid will have 3*3 = 9 parameter settings for CrossValidator to choose from.
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [80,90,100]) \
    .addGrid(rf.maxDepth, [3, 5]) \
    .build()

pipeline = Pipeline(stages=[rf])

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=4)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(trantf)

# Make predictions on test documents. cvModel uses the best model found (lrModel).
prediction = cvModel.transform(testtf)
selected = prediction.select( "content", "label", "prediction")
selected.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Apply the best Params and predict

In [10]:
# create the trainer and set its parameters
rf = RandomForestClassifier(numTrees=80,maxDepth=3)

# train the model
model = rf.fit(train)

# select example rows to display.
prediction = model.transform(test)

selected = prediction.select( "content", "label", "prediction")
selected.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+----------+
|             content|label|prediction|
+--------------------+-----+----------+
|Alan Rickman & Em...|  0.0|       1.0|
|I have seen this ...|  0.0|       1.0|
|In Los Angeles, t...|  0.0|       0.0|
|This film is bund...|  0.0|       0.0|
|I only comment on...|  0.0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows

### RF Model Evaluate

#### AUC-ROC 

In [11]:
# compute accuracy on the test set
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(selected)

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g " % (1.0 - accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy = 0.76184
Test Error = 0.23816

#### Precision-Recall

In [12]:
# compute accuracy on the test set
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderPR")
accuracy = evaluator.evaluate(selected)

print("Test Accuracy = %g" % (accuracy))
print("Test Error = %g " % (1.0 - accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test Accuracy = 0.71998
Test Error = 0.28002

In [13]:
# printBestParams(cvModel)

# savemodel(cvModel.bestModel,'s3://myprojectsentiment/train_model/RF_tf')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Multilayer Perceptron classifier

In [14]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

# specify layers for the neural network:
# input layer of size 500 (features), four intermediate layer
# and output of size 2 (classes)
layers = [1500, 128, 32, 8, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

# train the model
model = trainer.fit(train)

# compute accuracy on the test set
result = model.transform(test)

predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Test set accuracy = 0.79236

## Save Result

In [ ]:
import datetime

# write prediction file with timestamp file name
def savePrediction(lines, path):
    to_day = datetime.datetime.today()
    to_day = str(to_day)[0:19]
    to_day = re.sub("[^0-9a-zA-Z$]+","_",to_day)
    file_path = path + to_day
    file_path.strip()
    lines.coalesce(1).write.csv(file_path, header=True)
    

In [ ]:
savePrediction(predictionAndLabels,'s3://janev/output/project/')